# TAS for K8S

## Provision cluster

### Login to Tanzu Mission Control

In [1]:
source ../config.sh

tmc login --no-configure --name tanzu-demo

i $TMC_API_TOKEN is set 
√ Successfully created context tanzu-demo, to manage your contexts run `tmc system context -h`


### Create cluster

In [ ]:
source ../config.sh

tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 5 --instance-type m5.xlarge --group ${GROUP} --name ${ALPHA_CLUSTER}

### Monitor creation progress

In [ ]:
tmc cluster list --group bush

### Add cluster to kubeconfig file

In [ ]:
source ../config.sh

KUBECONFIG_ALPHA=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} >> ${KUBECONFIG_ALPHA}

if [ $? -eq 0 ]; then
    KUBECONFIG=${KUBECONFIG_ALPHA} kubectl config view --merge --flatten > ~/.kube/config
    rm ${KUBECONFIG_ALPHA}
fi

### Update cluster permissions

In [ ]:
source ../config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

### Delete clusters

In [16]:
source ../config.sh

tmc cluster delete ${ALPHA_CLUSTER}

√ Cluster is being deleted 


## [Install](https://github.com/cloudfoundry/cf-for-k8s) TAS for K8S

### Configure

In [ ]:
source ../config.sh

pushd ~/repos
rm -rf ./cf-for-k8s

git clone https://github.com/cloudfoundry/cf-for-k8s.git
cd cf-for-k8s

./hack/generate-values.sh -d cf.jb-cloud.com > ./cf-values.yml

# configure Docker Hub as image registry
echo "app_registry:
   hostname: https://index.docker.io/v1/
   repository_prefix: \"$DOCKERHUB_USERNAME\"
   username: \"$DOCKERHUB_USERNAME\"
   password: \"$DOCKERHUB_PASSWORD\"" >> ./cf-values.yml

ytt -f config -f ./cf-values.yml > ./cf-for-k8s-rendered.yml

popd

## Deploy

In [ ]:
source ../config.sh

kubectl config use-context ${ALPHA_CLUSTER}

pushd ~/repos/cf-for-k8s

kapp deploy -a cf -f ./cf-for-k8s-rendered.yml -y

popd

In [ ]:
kubectl get svc -n istio-system istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[*].hostname}'

```
# sample A record in Google cloud DNS. The IP address below is the address of Ingress gateway's external IP
Domain         Record Type  TTL  IP Address
*.<cf-domain>  A            30   35.111.111.111
```

Create CNAME for AWS load balancer:
Map *.cf to AWS load balancer hostname

```
watch -n 5 nslookup api.cf.jb-cloud.com
```

### Validate Deployment

In [13]:
curl -k https://api.cf.jb-cloud.com/v2/info

{"name":"","build":"","support":"","version":0,"description":"","authorization_endpoint":"https://login.cf.jb-cloud.com","token_endpoint":"https://uaa.cf.jb-cloud.com","min_cli_version":"","min_recommended_cli_version":"","app_ssh_endpoint":"TODO.TODO","app_ssh_host_key_fingerprint":"placeholder","app_ssh_oauth_client":"placeholder","doppler_logging_endpoint":"wss://doppler.cf.jb-cloud.com:443","api_version":"2.152.0","osbapi_version":"2.15","routing_endpoint":"https://api.cf.jb-cloud.com/routing"}

In [14]:
cf api --skip-ssl-validation https://api.cf.jb-cloud.com
cf auth admin wdgpwjbwa3lpucddnxac

Setting api endpoint to https://api.cf.jb-cloud.com...
OK

api endpoint:   https://api.cf.jb-cloud.com
api version:    3.87.0
API endpoint: https://api.cf.jb-cloud.com
Authenticating...
OK

Use 'cf target' to view or set your target org and space.


In [15]:
cf create-org test-org
cf create-space -o test-org test-space
cf target -o test-org -s test-space

Creating org test-org as admin...
OK

Assigning role OrgManager to user admin in org test-org...
OK

TIP: Use 'cf target -o "test-org"' to target new org
Creating space test-space in org test-org as admin...
OK

Assigning role SpaceManager to user admin in org test-org / space test-space as admin...
OK

Assigning role SpaceDeveloper to user admin in org test-org / space test-space as admin...
OK

TIP: Use 'cf target -o "test-org" -s "test-space"' to target new space
api endpoint:   https://api.cf.jb-cloud.com
api version:    2.152.0
user:           admin
org:            test-org
space:          test-space


In [ ]:
cf push test-node-app -p tests/smoke/assets/test-node-app

In [ ]:
# install metrics server
kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/download/v0.3.6/components.yaml

In [ ]:
# Assuming that you ran `kapp deploy -a cf...`
kapp delete -a cf

In [ ]:
openssl s_client -connect api.tas.jb-cloud.com:443 -msg